# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
%pip install hvplot
%pip install gmaps
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Wayne He\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Wayne He\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Wayne He\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kruisfontein,-34.0033,24.7314,9.57,72,88,0.38,ZA,1725859608
1,1,port-aux-francais,-49.3500,70.2167,3.32,80,35,10.61,TF,1725859593
2,2,qaqortoq,60.7167,-46.0333,5.22,63,100,2.85,GL,1725859608
3,3,adamstown,-25.0660,-130.1015,19.18,84,99,13.39,PN,1725859608
4,4,iqaluit,63.7506,-68.5145,3.85,64,0,4.63,CA,1725859608


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)

# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp0 = city_data_df.loc[city_data_df['Max Temp']<27,:]
city_temp = city_temp0.loc[city_temp0['Max Temp']>21,:]
city_wind = city_temp.loc[city_temp['Wind Speed']<4.5,:]
city_cloud = city_wind.loc[city_wind['Cloudiness']==0,:]
city_cloud
# Drop any rows with null values
city_ideal = city_cloud.dropna()
# Display sample data
city_ideal

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
125,125,newman,37.3138,-121.0208,25.58,52,0,1.34,US,1725859617
162,162,ungsang-nodongjagu,42.3578,130.4622,21.23,66,0,3.64,KP,1725859619
170,170,kailua-kona,19.6406,-155.9956,26.98,77,0,1.54,US,1725859620
233,233,tsiombe,-25.3000,45.4833,25.89,53,0,0.71,MG,1725859624
352,352,santanyi,39.3546,3.1291,21.75,78,0,4.15,ES,1725859632
356,356,hami,42.8000,93.4500,23.54,17,0,2.53,CN,1725859633
378,378,tupaciguara,-18.5922,-48.7050,21.71,31,0,1.96,BR,1725859635
394,394,brandon,27.9378,-82.2859,26.93,93,0,0.89,US,1725859637
410,410,isfahan,32.6572,51.6776,23.80,21,0,1.46,IR,1725859638
463,463,big spring,32.2504,-101.4787,21.80,59,0,3.09,US,1725859695


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
125,newman,US,37.3138,-121.0208,52,
162,ungsang-nodongjagu,KP,42.3578,130.4622,66,
170,kailua-kona,US,19.6406,-155.9956,77,
233,tsiombe,MG,-25.3000,45.4833,53,
352,santanyi,ES,39.3546,3.1291,78,
356,hami,CN,42.8000,93.4500,17,
378,tupaciguara,BR,-18.5922,-48.7050,31,
394,brandon,US,27.9378,-82.2859,93,
410,isfahan,IR,32.6572,51.6776,21,
463,big spring,US,32.2504,-101.4787,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
newman - nearest hotel: No hotel found
ungsang-nodongjagu - nearest hotel: No hotel found
kailua-kona - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
santanyi - nearest hotel: No hotel found
hami - nearest hotel: No hotel found
tupaciguara - nearest hotel: No hotel found
brandon - nearest hotel: No hotel found
isfahan - nearest hotel: No hotel found
big spring - nearest hotel: No hotel found
santa cruz - nearest hotel: No hotel found
taco pozo - nearest hotel: No hotel found
nazarabad - nearest hotel: No hotel found
yeppoon - nearest hotel: No hotel found
toliara - nearest hotel: No hotel found
silsbee - nearest hotel: No hotel found
trinidad - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
125,newman,US,37.3138,-121.0208,52,No hotel found
162,ungsang-nodongjagu,KP,42.3578,130.4622,66,No hotel found
170,kailua-kona,US,19.6406,-155.9956,77,No hotel found
233,tsiombe,MG,-25.3000,45.4833,53,No hotel found
352,santanyi,ES,39.3546,3.1291,78,No hotel found
356,hami,CN,42.8000,93.4500,17,No hotel found
378,tupaciguara,BR,-18.5922,-48.7050,31,No hotel found
394,brandon,US,27.9378,-82.2859,93,No hotel found
410,isfahan,IR,32.6572,51.6776,21,No hotel found
463,big spring,US,32.2504,-101.4787,59,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)